In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',', dtype='float32')

In [4]:
x_data = data[:, 0:2]
y_data = data[:,2:]

global_step = tf.Variable(0, trainable = False, name="global_step")
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1, 1), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1, 1), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1, 1), name='W3')
    model = tf.matmul(L2, W3)
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

In [19]:
saver = tf.train.Saver(tf.global_variables())

with tf.Session() as sess:
    # 체크포인트가 존재하는지 검사
    ckpt = tf.train.get_checkpoint_state('./model_tensorboard')
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        # 체크포인트가 존재하는 경우, 복원
        # global_step 값도 복원됨
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        # 체크포인트가 존재하지 않는 경우, 젂역 변수 초기화
        sess.run(tf.global_variables_initializer())
    
    # summary 기록 준비
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)
    
     # 학습
    feed_dict = {X: x_data, Y: y_data}
    for step in range(100):
        sess.run(train_op, feed_dict = feed_dict)
        print('Step: {}, Cost: {:.3f}'.format(
                sess.run(global_step),
                sess.run(cost, feed_dict=feed_dict)))
    # 추적값 저장
    summary = sess.run(merged, feed_dict=feed_dict)
    writer.add_summary(summary, global_step=sess.run(global_step))
    
    # 체크포인트 저장
    saver.save(sess, './model_tensorboard/dnn.ckpt', global_step=global_step)
    
    # 학습 결과 확인
    prediction = tf.argmax(model, axis=1)
    target = tf.argmax(Y, axis=1)
    print("예측값 : ", sess.run(prediction, feed_dict={X : x_data}))
    print("실제값 : ", sess.run(target, feed_dict={Y : y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도 : {:.2f}'.format( sess.run(accuracy* 100, feed_dict=feed_dict)) )

INFO:tensorflow:Restoring parameters from ./model_tensorboard\dnn.ckpt-100
Step: 101, Cost: 1.976
Step: 102, Cost: 2.001
Step: 103, Cost: 2.030
Step: 104, Cost: 2.067
Step: 105, Cost: 2.098
Step: 106, Cost: 2.125
Step: 107, Cost: 2.163
Step: 108, Cost: 2.208
Step: 109, Cost: 2.253
Step: 110, Cost: 2.288
Step: 111, Cost: 2.322
Step: 112, Cost: 2.358
Step: 113, Cost: 2.400
Step: 114, Cost: 2.441
Step: 115, Cost: 2.478
Step: 116, Cost: 2.524
Step: 117, Cost: 2.569
Step: 118, Cost: 2.616
Step: 119, Cost: 2.659
Step: 120, Cost: 2.695
Step: 121, Cost: 2.735
Step: 122, Cost: 2.774
Step: 123, Cost: 2.812
Step: 124, Cost: 2.857
Step: 125, Cost: 2.888
Step: 126, Cost: 2.935
Step: 127, Cost: 2.989
Step: 128, Cost: 3.051
Step: 129, Cost: 3.106
Step: 130, Cost: 3.151
Step: 131, Cost: 3.192
Step: 132, Cost: 3.230
Step: 133, Cost: 3.270
Step: 134, Cost: 3.306
Step: 135, Cost: 3.342
Step: 136, Cost: 3.382
Step: 137, Cost: 3.425
Step: 138, Cost: 3.488
Step: 139, Cost: 3.550
Step: 140, Cost: 3.606
Step: